In [1]:
# Check to see if we're using a GPU
!nvidia-smi

Fri Oct 13 23:06:59 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 101 food classes: working with less data

Our goal is to beath the original Food101 paper with 10% of the training data.

In [2]:
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip

--2023-10-13 23:07:00--  https://storage.googleapis.com/ztm_tf_course/food_vision/101_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.234.207, 142.251.172.207, 172.217.212.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.234.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1625420029 (1.5G) [application/zip]
Saving to: ‘101_food_classes_10_percent.zip’

101_food_classes_10 100%[===================>]   1.51G  89.0MB/s    in 16s     

2023-10-13 23:07:15 (99.4 MB/s) - ‘101_food_classes_10_percent.zip’ saved [1625420029/1625420029]



In [3]:
import zipfile
zip_ref = zipfile.ZipFile("101_food_classes_10_percent.zip")
zip_ref.extractall()
zip_ref.close()

In [4]:
# How many images in each folder?
import os

# Walk through -1 percent data directory and list number of files
for dirpath, dirnames, filenames in os.walk("101_food_classes_10_percent"):
  print(f"There are {len(dirnames)} directories and {len(filenames)} images in `{dirpath}`")

There are 2 directories and 0 images in `101_food_classes_10_percent`
There are 101 directories and 0 images in `101_food_classes_10_percent/train`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/french_toast`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/frozen_yogurt`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/baklava`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/donuts`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/french_onion_soup`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/bruschetta`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/ramen`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/cheesecake`
There are 0 directories and 75 images in `101_food_classes_10_percent/train/hot_dog`
There are 0 directories and 75 images in `101_food_classes_10_p

In [5]:
train_dir = "101_food_classes_10_percent/train/"
test_dir = "101_food_classes_10_percent/test/"

In [6]:
# Setup data inputs
import tensorflow as tf
IMG_SIZE = (224, 224)

train_data_all_10_percent = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                                label_mode="categorical",
                                                                                image_size=IMG_SIZE)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                label_mode="categorical",
                                                                image_size=IMG_SIZE,
                                                                shuffle=False)  # don't shuffle test data for prediction analysis

Found 7575 files belonging to 101 classes.
Found 25250 files belonging to 101 classes.


## Train a big dog model with transfer learning on 10% of 101 food classes

In [7]:
checkpoint_path = "101_classes_10_percent_data_model_checkpoint"
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                         save_weights_only=True,
                                                         monitor="val_accuracy",
                                                         save_best_only=True)

In [8]:
# Create data augmentation layer to incorporate it righ into the model
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.models import Sequential

# Setup data augmentation
data_augmentation = Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    # preprocessing.Rescaling(1/255.)   # rescale inputs of images to between 0 and 1, required for models like ResNet50
])

In [9]:
# Setup the base model and freeze its layer (this will extract features)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Set up model architecture with trainable top layers
inputs = layers.Input(shape=(224, 224, 3), name="input_layer")
x = data_augmentation(inputs) # augment images (only happens during training phase)
x = base_model(x, training=False) # put the base model in inference mode so weights which need to stay frozen, stay frozen
x = layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
outputs = layers.Dense(len(train_data_all_10_percent.class_names), activation="softmax", name="output_layer")(x)
model = tf.keras.Model(inputs, outputs)

16705208/16705208 [==============================] - 0s 0us/step


In [10]:
# Get a summary of model we've created
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, None, None, 3)     0         
                                                                 
 efficientnetb0 (Functional  (None, None, None, 1280   4049571   
 )                           )                                   
                                                                 
 global_avg_pool_layer (Glo  (None, 1280)              0         
 balAveragePooling2D)                                            
                                                                 
 output_layer (Dense)        (None, 101)               129381    
                                                                 
Total params: 4178952 (15.94 MB)
Trainable params: 129381 (50

In [11]:
# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

In [12]:
# Fit the model
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs=5,
                                           steps_per_epoch=len(train_data_all_10_percent),
                                           validation_data=test_data,
                                           validation_steps=int(0.15 * len(test_data)),
                                           # Track model training logs
                                           callbacks=[checkpoint_callback])

Epoch 1/5
237/237 [==============================] - 119s 433ms/step - loss: 3.3724 - accuracy: 0.2756 - val_loss: 2.5413 - val_accuracy: 0.4203
Epoch 2/5
237/237 [==============================] - 78s 328ms/step - loss: 2.2005 - accuracy: 0.5005 - val_loss: 2.0748 - val_accuracy: 0.4897
Epoch 3/5
237/237 [==============================] - 69s 287ms/step - loss: 1.8124 - accuracy: 0.5686 - val_loss: 1.8889 - val_accuracy: 0.5238
Epoch 4/5
237/237 [==============================] - 65s 274ms/step - loss: 1.6042 - accuracy: 0.6110 - val_loss: 1.8189 - val_accuracy: 0.5281
Epoch 5/5
237/237 [==============================] - 61s 255ms/step - loss: 1.4451 - accuracy: 0.6498 - val_loss: 1.7572 - val_accuracy: 0.5408


## Fine-tuning

In [14]:
# Unfreeze all of the layer in the base model
base_model.trainable = True

# Freeze every layer except the last 5
for layer in base_model.layers[:-5]:
  layer.trainable=False

In [15]:
# recompile model with lower learning (it's typically best practice to lower the larning rate when fine-tuning)
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              metrics=["accuracy"])

In [16]:
# What layers in the model are trainable?
for layer in model.layers:
  print(layer.name, layer.trainable)

input_layer True
sequential True
efficientnetb0 True
global_avg_pool_layer True
output_layer True


In [17]:
# Check which layers are trainable in our base model
for layer_number, layer in enumerate(model.layers[2].layers):
  print(layer_number, layer.name, layer.trainable)

0 input_1 False
1 rescaling False
2 normalization False
3 rescaling_1 False
4 stem_conv_pad False
5 stem_conv False
6 stem_bn False
7 stem_activation False
8 block1a_dwconv False
9 block1a_bn False
10 block1a_activation False
11 block1a_se_squeeze False
12 block1a_se_reshape False
13 block1a_se_reduce False
14 block1a_se_expand False
15 block1a_se_excite False
16 block1a_project_conv False
17 block1a_project_bn False
18 block2a_expand_conv False
19 block2a_expand_bn False
20 block2a_expand_activation False
21 block2a_dwconv_pad False
22 block2a_dwconv False
23 block2a_bn False
24 block2a_activation False
25 block2a_se_squeeze False
26 block2a_se_reshape False
27 block2a_se_reduce False
28 block2a_se_expand False
29 block2a_se_excite False
30 block2a_project_conv False
31 block2a_project_bn False
32 block2b_expand_conv False
33 block2b_expand_bn False
34 block2b_expand_activation False
35 block2b_dwconv False
36 block2b_bn False
37 block2b_activation False
38 block2b_se_squeeze False
39

In [18]:
# Fine tune for 5 more epochs
fine_tune_epochs = 10    # model has already done 5 epochs (feature extraction), this is the total number of epochs we're after (5 + 5 = 10)

# Refit the model (same as model except with more trainable layers)
history_all_classes_10_percent_fine_tune= model.fit(train_data_all_10_percent,
                                            epochs=fine_tune_epochs,
                                            validation_data=test_data,
                                            validation_steps=int(0.15 * len(test_data)),
                                            initial_epoch=history_all_classes_10_percent.epoch[-1],  # start training from previous last epoch
                                            callbacks=[checkpoint_callback] )

Epoch 5/10
237/237 [==============================] - 69s 251ms/step - loss: 1.2137 - accuracy: 0.6745 - val_loss: 1.6984 - val_accuracy: 0.5490
Epoch 6/10
237/237 [==============================] - 53s 222ms/step - loss: 1.1006 - accuracy: 0.7064 - val_loss: 1.6854 - val_accuracy: 0.5548
Epoch 7/10
237/237 [==============================] - 49s 205ms/step - loss: 1.0197 - accuracy: 0.7266 - val_loss: 1.6929 - val_accuracy: 0.5508
Epoch 8/10
237/237 [==============================] - 46s 193ms/step - loss: 0.9556 - accuracy: 0.7455 - val_loss: 1.7188 - val_accuracy: 0.5482
Epoch 9/10
237/237 [==============================] - 47s 197ms/step - loss: 0.8915 - accuracy: 0.7592 - val_loss: 1.6875 - val_accuracy: 0.5540
Epoch 10/10
237/237 [==============================] - 46s 189ms/step - loss: 0.8277 - accuracy: 0.7777 - val_loss: 1.7214 - val_accuracy: 0.5469
